Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

# Logistic Model

In [5]:
def prediction(X1, W1, b1):
    logits = tf.matmul(X1, W1) + b1
    return tf.nn.softmax(logits)

In [42]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variable
    W1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    b1 = tf.Variable(tf.zeros([num_labels]))
    
    # Train computation
    logits = tf.matmul(tf_train_dataset, W1) + b1
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #add l2_loss to loss
    loss += beta * tf.nn.l2_loss(W1)
    
    # Optimization
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions on training, validation and test data
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = prediction(tf_valid_dataset, W1, b1)
    test_prediction = prediction(tf_test_dataset, W1, b1)

In [43]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 24.383547
Minibatch accuracy: 9.4%
Validation accuracy: 9.3%
Minibatch loss at step 500: 2.851146
Minibatch accuracy: 74.2%
Validation accuracy: 76.8%
Minibatch loss at step 1000: 1.891115
Minibatch accuracy: 76.6%
Validation accuracy: 79.0%
Minibatch loss at step 1500: 1.121311
Minibatch accuracy: 86.7%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 0.883754
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.745089
Minibatch accuracy: 82.8%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.823692
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Test accuracy: 89.0%


# Neutral network with relu

In [7]:
def prediction_NN(X1,W1,b1,W2,b2):
    Y1 = tf.matmul(X1, W1) + b1
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    return tf.nn.softmax(Y2)

In [8]:
batch_size = 128
hidden_nodes = 1024
beta = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  b1 = tf.Variable(tf.zeros([hidden_nodes]))
  W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  # Go through RELU   
  Y1 = tf.matmul(tf_train_dataset, W1) + b1 
  X2 = tf.nn.relu(Y1)
  logits = tf.matmul(X2, W2) + b2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  #add l2_loss to loss
  loss += beta*(tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction_NN(tf_valid_dataset,W1,b1,W2,b2)
  test_prediction = prediction_NN(tf_test_dataset,W1,b1,W2,b2)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 612.307251
Minibatch accuracy: 12.5%
Validation accuracy: 34.1%
Minibatch loss at step 500: 207.726242
Minibatch accuracy: 76.6%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 114.563728
Minibatch accuracy: 77.3%
Validation accuracy: 81.9%
Minibatch loss at step 1500: 69.390427
Minibatch accuracy: 83.6%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 41.214211
Minibatch accuracy: 89.8%
Validation accuracy: 85.6%
Minibatch loss at step 2500: 25.191715
Minibatch accuracy: 89.8%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 15.431751
Minibatch accuracy: 89.8%
Validation accuracy: 87.0%
Test accuracy: 93.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [5]:
def prediction_NN(X1,W1,b1,W2,b2):
    Y1 = tf.matmul(X1, W1) + b1
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    return tf.nn.softmax(Y2)

In [6]:
batch_size = 16
train_size = 50
hidden_nodes = 1024
beta = 0.001
# Make train data set smaller
mini_train_dataset = train_dataset[:train_size]
mini_train_labels = train_labels[:train_size]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  b1 = tf.Variable(tf.zeros([hidden_nodes]))
  W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  # Go through RELU   
  Y1 = tf.matmul(tf_train_dataset, W1) + b1 
  X2 = tf.nn.relu(Y1)
  logits = tf.matmul(X2, W2) + b2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  #add l2_loss to loss
  loss += beta*(tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction_NN(tf_valid_dataset,W1,b1,W2,b2)
  test_prediction = prediction_NN(tf_test_dataset,W1,b1,W2,b2)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (mini_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = mini_train_dataset[offset:(offset + batch_size), :]
    batch_labels = mini_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 621.149902
Minibatch accuracy: 6.2%
Validation accuracy: 29.0%
Minibatch loss at step 500: 198.497559
Minibatch accuracy: 100.0%
Validation accuracy: 52.1%
Minibatch loss at step 1000: 120.379799
Minibatch accuracy: 100.0%
Validation accuracy: 52.2%
Minibatch loss at step 1500: 73.005013
Minibatch accuracy: 100.0%
Validation accuracy: 52.2%
Minibatch loss at step 2000: 44.274250
Minibatch accuracy: 100.0%
Validation accuracy: 52.2%
Minibatch loss at step 2500: 26.850342
Minibatch accuracy: 100.0%
Validation accuracy: 52.3%
Minibatch loss at step 3000: 16.283552
Minibatch accuracy: 100.0%
Validation accuracy: 52.4%
Test accuracy: 56.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [19]:
def prediction_NN_Drop(X1,W1,b1,W2,b2):
    Y1 = tf.matmul(X1, W1) + b1
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    return tf.nn.softmax(Y2)

In [20]:
batch_size = 128
hidden_nodes = 1024
beta = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_keep_prob = tf.placeholder(tf.float32)

  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  b1 = tf.Variable(tf.zeros([hidden_nodes]))
  W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Go through RELU   
  Y1 = tf.matmul(tf_train_dataset, W1) + b1 
  X2 = tf.nn.relu(Y1)
  
  # Apply drop out
  X2_drop = tf.nn.dropout(X2, tf_keep_prob)
    
  logits = tf.matmul(X2_drop, W2) + b2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  #add l2_loss to loss
  loss += beta*(tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction_NN_Drop(tf_valid_dataset,W1,b1,W2,b2)
  test_prediction = prediction_NN_Drop(tf_test_dataset,W1,b1,W2,b2)

In [21]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 862.458923
Minibatch accuracy: 6.2%
Validation accuracy: 27.8%
Minibatch loss at step 500: 221.746628
Minibatch accuracy: 74.2%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 120.857948
Minibatch accuracy: 71.9%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 71.458412
Minibatch accuracy: 82.0%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 41.557682
Minibatch accuracy: 85.2%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 25.345715
Minibatch accuracy: 84.4%
Validation accuracy: 85.2%
Minibatch loss at step 3000: 15.544609
Minibatch accuracy: 81.2%
Validation accuracy: 85.6%
Minibatch loss at step 3500: 9.694987
Minibatch accuracy: 85.2%
Validation accuracy: 86.4%
Minibatch loss at step 4000: 5.873853
Minibatch accuracy: 91.4%
Validation accuracy: 86.5%
Minibatch loss at step 4500: 3.957057
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Minibatch loss at step 5000: 2.519515
Minibatch accuracy: 85.9%
Validatio

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


# 2 hidden layer

In [15]:
def prediction_NN_Drop(X1,W1,b1,W2,b2,W3,b3):
    Y1 = tf.matmul(X1, W1) + b1
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    X3 = tf.nn.relu(Y2)
    Y3 = tf.matmul(X3, W3) + b3
    return tf.nn.softmax(Y3)

###########################################################

batch_size = 128
hidden_nodes_layer_1 = 1024
hidden_nodes_layer_2 = 512
beta = 1e-4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_keep_prob = tf.placeholder(tf.float32)

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_layer_1],stddev=0.1))
  b1 = tf.Variable(tf.zeros([hidden_nodes_layer_1]))
  W2 = tf.Variable(tf.truncated_normal([hidden_nodes_layer_1, hidden_nodes_layer_2],stddev=0.1))
  b2 = tf.Variable(tf.zeros([hidden_nodes_layer_2]))
  W3 = tf.Variable(tf.truncated_normal([hidden_nodes_layer_2, num_labels],stddev=0.1))
  b3 = tf.Variable(tf.zeros([num_labels]))
  
  # Go through RELU & dropout  
  Y1 = tf.matmul(tf_train_dataset, W1) + b1

  X2 = tf.nn.relu(Y1)
  X2_drop = tf.nn.dropout(X2, tf_keep_prob)
  Y2 = tf.matmul(X2_drop, W2) + b2

  X3 = tf.nn.relu(Y2)
  X3_drop = tf.nn.dropout(X3, tf_keep_prob)  
  logits = tf.matmul(X3_drop, W3) + b3

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  #add l2_loss to loss
  loss += beta*(tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3))  
  
  # Optimizer & apply exponeital decay of learning rate
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction_NN_Drop(tf_valid_dataset,W1,b1,W2,b2,W3,b3)
  test_prediction = prediction_NN_Drop(tf_test_dataset,W1,b1,W2,b2,W3,b3)

############################################################################################

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob: 0.8}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 6.326581
Minibatch accuracy: 7.0%
Validation accuracy: 18.2%
Minibatch loss at step 500: 1.136475
Minibatch accuracy: 80.5%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 1.150054
Minibatch accuracy: 78.1%
Validation accuracy: 86.5%
Minibatch loss at step 1500: 0.994613
Minibatch accuracy: 84.4%
Validation accuracy: 87.4%
Minibatch loss at step 2000: 0.777927
Minibatch accuracy: 89.8%
Validation accuracy: 88.0%
Minibatch loss at step 2500: 0.858524
Minibatch accuracy: 89.1%
Validation accuracy: 88.2%
Minibatch loss at step 3000: 0.806565
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Test accuracy: 94.0%


# 3 hidden layer 

In [10]:
def prediction_NN_Drop(X1,W1,b1,W2,b2,W3,b3,W4,b4):
    Y1 = tf.matmul(X1, W1) + b1
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    X3 = tf.nn.relu(Y2)
    Y3 = tf.matmul(X3, W3) + b3
    X4 = tf.nn.relu(Y3)
    Y4 = tf.matmul(X4, W4) + b4
    return tf.nn.softmax(Y4)

###########################################################

batch_size = 128
hidden_nodes_layer_1 = 1024
hidden_nodes_layer_2 = 512
hidden_nodes_layer_3 = 128
beta = 1e-4
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_keep_prob = tf.placeholder(tf.float32)

  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_layer_1],stddev=0.1))
  b1 = tf.Variable(tf.zeros([hidden_nodes_layer_1]))
  W2 = tf.Variable(tf.truncated_normal([hidden_nodes_layer_1, hidden_nodes_layer_2],stddev=0.1))
  b2 = tf.Variable(tf.zeros([hidden_nodes_layer_2]))
  W3 = tf.Variable(tf.truncated_normal([hidden_nodes_layer_2, hidden_nodes_layer_3],stddev=0.1))
  b3 = tf.Variable(tf.zeros([hidden_nodes_layer_3]))
  W4 = tf.Variable(tf.truncated_normal([hidden_nodes_layer_3, num_labels],stddev=0.1))
  b4 = tf.Variable(tf.zeros([num_labels]))
  
  # Go through RELU   
  Y1 = tf.matmul(tf_train_dataset, W1) + b1 
  X2 = tf.nn.relu(Y1)
  X2_drop = tf.nn.dropout(X2, tf_keep_prob)

  Y2 = tf.matmul(X2_drop, W2) + b2
  X3 = tf.nn.relu(Y2)
  X3_drop = tf.nn.dropout(X3, tf_keep_prob)
    
  Y3 = tf.matmul(X3_drop, W3) + b3
  X4 = tf.nn.relu(Y3)
  X4_drop = tf.nn.dropout(X4, tf_keep_prob)
    
  logits = tf.matmul(X4_drop, W4) + b4
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  #add l2_loss to loss
  loss += beta*(tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3) + tf.nn.l2_loss(W4))  
  
  # Optimizer & apply exponeital decay of learning rate
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction_NN_Drop(tf_valid_dataset,W1,b1,W2,b2,W3,b3,W4,b4)
  test_prediction = prediction_NN_Drop(tf_test_dataset,W1,b1,W2,b2,W3,b3,W4,b4)

############################################################################################

num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob: 0.8}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 5.291780
Minibatch accuracy: 3.9%
Validation accuracy: 17.5%
Minibatch loss at step 5000: 0.666690
Minibatch accuracy: 93.0%
Validation accuracy: 89.3%
Minibatch loss at step 10000: 0.487079
Minibatch accuracy: 95.3%
Validation accuracy: 90.7%
Minibatch loss at step 15000: 0.454043
Minibatch accuracy: 94.5%
Validation accuracy: 91.0%
Minibatch loss at step 20000: 0.546536
Minibatch accuracy: 92.2%
Validation accuracy: 91.3%
Minibatch loss at step 25000: 0.367313
Minibatch accuracy: 96.9%
Validation accuracy: 91.6%
Minibatch loss at step 30000: 0.337158
Minibatch accuracy: 98.4%
Validation accuracy: 91.6%
Test accuracy: 96.2%


In [ ]:
#Attention: if set the loss at the beginning is too large (this case > 10) -> get NaN;
#changing the parameters to receive a smaller one will help.